# **Montando o Drive**

In [ ]:
# Coloca os arquivos do google drive disponíveis para serem usados pelo colab

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# **Carregando o Arquivo das Bounding Boxes a ser Usado**

In [ ]:
import torch

bboxes_xywh = torch.load('/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/bounding_boxes_xywh.pt', map_location=torch.device('cpu'))

# **Cálculo das Áreas das Bounding Box**

In [4]:
from PIL import Image, ImageDraw, ImageFont
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import csv

folder_path = "/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/YOLOv8x_tuned/Resultados/Teste/inferencias"
image_size = 416 * 416

image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]

image_paths.sort()

output_folder = '/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/imagens_areas_calculadas'

coverage_values = []

# Iterate through the list of tensors and image paths
for i, (tensor, image_path) in enumerate(zip(bboxes_xywh, image_paths)):
    total_area = 0
    # Calculate the total area of bounding boxes
    for bbox in tensor:
        x, y, w, h = bbox
        area = w * h
        total_area += area
    
    # Calculate the percentage of the image covered by bounding boxes
    coverage = (total_area / image_size) * 100
    coverage_values.append(coverage)
    
    # Load the image using OpenCV
    image = cv2.imread(image_path)
       
    # Convert the OpenCV image to a PIL Image
    pil_image = Image.fromarray(image)

    # Overlay the percentage on the image using Pillow
    draw = ImageDraw.Draw(pil_image)
    font_path = "/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/arial.ttf"  # Replace with the path to your font file
    font_size = 18  # Adjust the font size
    font = ImageFont.truetype(font_path, font_size)
    text = f"Área Cultivada: {coverage:.2f}%"
    text_position = (5, 5)  # Adjust the position of the text (x, y)
    text_color = (255, 255, 255)  # White text color

    draw.text(text_position, text, fill=text_color, font=font)
    
    image = np.array(pil_image)

    # Save the image with the overlay text
    output_image_path = os.path.join(output_folder, f"image_{i}_area.jpg")
    cv2.imwrite(output_image_path, image)

    # Display the image
    cv2_imshow(image)

output_csv_path = "/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/coverage_values.csv"

with open(output_csv_path, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Image", "Coverage"])
    for i, coverage in enumerate(coverage_values):
        csv_writer.writerow([f"image_{i}_area.jpg", float(coverage.item())])
    
# Wait for a key press and close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.